> *Processo de ingestão da Tabela XPTO da API do IBGE SIDRA*

| Camada      | Entidade              |
| ----------  | ----------            |
| raw         | pib_ibge.json      |



> *Histórico de Alterações*

| Time      | Projeto               | Autor                                   | Data               | Descrição                                                              |
|-----------|-----------------------|-----------------------------------------|--------------------|------------------------------------------------------------------------|
| AL DataLab       | Data Engineer Challenge     | Ana/Lucas                     |   08/12/2025       | Desenvolvimento inicial do notebook                                    |
| AL DataLab       | Data Engineer Challenge     | xxxxxxxxxxxxxxxxx                       |   xx/xx/xxxx       | xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx                                     |

In [0]:
catalog  = 'dataengineerchallenge'
schema   = '2_trusted'
table    = 'tru_ibge_pib'

In [0]:
%run "/Workspace/Users/al.datalab.global@gmail.com/desafio_ibge/04 - UTILS/import_libs"

In [0]:
%run "/Workspace/Users/al.datalab.global@gmail.com/desafio_ibge/04 - UTILS/funcs"

In [0]:
# DEFINE A VARIÁVEL DT_CARGA
dt_carga = datetime.now().strftime('%Y-%m-%d')

In [0]:
base_path = "/Volumes/dataengineerchallenge/1_raw/raw_volume"
file_path = f"{base_path}/{dt_carga}/pib_ibge.json"

In [0]:
# LEITURA DO JSON QUE ESTÁ DATA LAKE/VOLUME
df = spark.read.json(file_path)

header_row = df.first()
novas_colunas_raw  = [str(c) for c in header_row]

novas_colunas = [sanitize_col_name(c) for c in novas_colunas_raw]

w = Window.orderBy(lit(1))

df_tmp = df.withColumn("rn", row_number().over(w))

df_sem_header = (
    df_tmp
    .filter("rn > 1")
    .drop("rn")
)

df_final = df_sem_header.toDF(*novas_colunas)


In [0]:
df_final.write \
  .format("delta") \
  .mode("overWrite") \
  .saveAsTable(f"{catalog}.{schema}.{table}")
